In [78]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import concurrent.futures

In [79]:
url = "https://www.mse.mk/mk/stats/symbolhistory/kmb"

In [80]:
response = requests.get(url)
html = response.text

In [81]:
soup = BeautifulSoup(html,"html.parser")

In [82]:
start_time = time.time()

In [83]:
sifri = soup.select("#Code > option")

In [84]:
companies = []
for s in sifri:
  if not any(char.isdigit() for char in s.text):
    companies.append(s.text)

In [85]:
len(companies)

165

In [86]:
base_url = "https://www.mse.mk/mk/stats/symbolhistory/"
print(companies[1].lower())

alk


In [87]:
from datetime import datetime, timedelta

In [88]:
current_date=datetime.now().date()

In [89]:
current_date_str = current_date.strftime("%d.%m.%Y")
current_date_str

'14.11.2024'

In [90]:
current_date_str=current_date_str.replace('-2024','')
razdeleno=current_date_str.split("-")
razdeleno

['14.11.2024']

In [91]:
date_365_days_ago = (datetime.now() - timedelta(days=365)).strftime("%d.%m.%Y")
date_365_days_ago

'15.11.2023'

In [92]:
curr_tmp=datetime.now().date().strftime("%d.%m.%Y")
print(curr_tmp)
for i in range(0,9):
  curr_date=datetime.strptime(curr_tmp, "%d.%m.%Y")
  year_ago_date = (curr_date - timedelta(days=365)).strftime("%d.%m.%Y")
  print(year_ago_date)
  curr_tmp=year_ago_date

14.11.2024
15.11.2023
15.11.2022
15.11.2021
15.11.2020
16.11.2019
16.11.2018
16.11.2017
16.11.2016
17.11.2015


In [93]:
import concurrent.futures
import requests
from datetime import datetime, timedelta
import pandas as pd
from bs4 import BeautifulSoup


def extract_all_elements_perYear(element, elem_Name):
    rows = element.select("#resultsTable > tbody > tr")
    if rows is None:
        return 0
    rows_dict = []
    name = elem_Name.upper()
    for row in rows:
        date = row.select_one("td:nth-child(1)").text
        cena_posledna = row.select_one("td:nth-child(2)").text
        max = row.select_one("td:nth-child(3)").text
        min = row.select_one("td:nth-child(4)").text
        prosecna_cena = row.select_one("td:nth-child(5)").text
        prom = row.select_one("td:nth-child(6)").text
        kolicina = row.select_one("td:nth-child(7)").text
        promet = row.select_one("td:nth-child(8)").text
        vkupno = row.select_one("td:nth-child(9)").text
        row_dict = {
            "Name": name,
            "Date": date,
            "PoslednaCena": cena_posledna,
            "Max": max,
            "Min": min,
            "ProsecnaCena": prosecna_cena,
            "%Prom": prom,
            "Kolicina": kolicina,
            "Promet": promet,
            "Vkupno": vkupno
        }
        rows_dict.append(row_dict)
    return rows_dict


def fetch_data_for_company_and_year(companie, year, curr_tmp):
    curr_date = datetime.strptime(curr_tmp, "%d.%m.%Y")
    year_ago_date = (curr_date - timedelta(days=365)).strftime("%d.%m.%Y")
    url_companie = base_url + companie.lower() + "?FromDate=" + year_ago_date + "&ToDate=" + curr_tmp
    response = requests.get(url_companie)
    soup = BeautifulSoup(response.text, "html.parser")
    myList = extract_all_elements_perYear(soup, companie)
    if myList == 0:
        return None
    return pd.DataFrame(myList), year_ago_date


frames = []
workers = 10
batch_size = 10
curr_tmp = datetime.now().date().strftime("%d.%m.%Y")


with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:
    futures = []


    for company in companies:
        for y in range(0, 9):
            futures.append(executor.submit(fetch_data_for_company_and_year, company, y, curr_tmp))


    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        if result:
            tmp_df, new_curr_tmp = result
            frames.append(tmp_df)
            curr_tmp = new_curr_tmp




In [94]:
result=pd.concat(frames)
result.to_csv("berza.csv",index=False)

In [95]:
result

,Name,Date,PoslednaCena,Max,Min,ProsecnaCena,%Prom,Kolicina,Promet,Vkupno
0,ADIN,14.11.2024,"1.650,00",,,"1.650,00","0,00",0,0,0
1,ADIN,13.11.2024,"1.650,00",,,"1.650,00","0,00",0,0,0
2,ADIN,12.11.2024,"1.650,00",,,"1.650,00","0,00",0,0,0
3,ADIN,11.11.2024,"1.650,00",,,"1.650,00","0,00",0,0,0
4,ADIN,08.11.2024,"1.650,00",,,"1.650,00","0,00",0,0,0
...,...,...,...,...,...,...,...,...,...,...
195,ZUAS,29.1.2024,"2.100,00",,,"2.100,00","0,00",0,0,0
196,ZUAS,26.1.2024,"2.100,00",,,"2.100,00","0,00",0,0,0
197,ZUAS,25.1.2024,"2.100,00",,,"2.100,00","0,00",0,0,0
198,ZUAS,24.1.2024,"2.100,00",,,"2.100,00","0,00",0,0,0


In [96]:
end_time = time.time()

In [97]:
print((end_time-start_time)/60 , 'minutes')

20.498690287272137 minutes
